# Project 1: Web scraping and basic summarization
*University of Ljubljana, Faculty for computer and information science* <br />
*Course: Introduction to data science*

In this Project you need to implement missing parts of this Jupyter notebook. All the code in the notebook must be reproducible and runnable, so include instructions for the environment setup or other specifics needed to run the notebook. The overview of the repository and setup should be put to README.md

The idea of this Project is to automatically retrieve structured data from pages [rtvslo.si](https://www.rtvslo.si) or [24ur.com](https://www.24ur.com). 

## Environment setup

Write instructions how to setup the environment to run this notebook, which libraries are installed, etc. Also provide installation sources.

`TODO: environment setup description`

In [3]:
# Load all the libraries needed for running the code chunks below

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
from selenium.webdriver.common.by import By



In [4]:
MAIN_URL = "https://www.rtvslo.si"
search_KEY = "Koronavirus"
PATH = r"chromedriver.exe"
n = 100 #max number of articles - some number

options = webdriver.ChromeOptions()
options.add_argument('window-size=2880,1800')
options.add_argument("--start-maximized") #so I can see it in fullscreen on laptop
driver = webdriver.Chrome(PATH, options=options)


driver.get(MAIN_URL)

print("Starting to get articles URLs from page: ",driver.title)

search = driver.find_element_by_id("header-search-input") #search for key word
search.send_keys(search_KEY)
search.send_keys(Keys.RETURN)

time.sleep(5)

i = 1 #page counter
j = 1
URLS = [] #list of all urls

while (len(URLS)< n - 11): #max number of articles
    articles = driver.find_elements_by_class_name("article-archive-item")
    time.sleep(1)
    print("I'm on page: {}".format(i))
    i = i + 1
    for article in articles:
        items = article.find_element_by_css_selector('div a')
        print(j, items.get_attribute('href'))
        j = j + 1
        time.sleep(1)
        URLS.append(items.get_attribute('href'))
    #print(URLS)   
    #print("########################")
   
    time.sleep(1)  
    driver.execute_script("window.scrollTo(0, 1800)") #scroll to bottom of the page
    time.sleep(1)
    
    paginator = driver.find_elements_by_css_selector("nav ul li")
    
    last_page = paginator[-1]
    last_page.click() #click to go to next page
    

with open('article_urls.txt', 'w') as f: #write URLs to file
    for url in URLS:
        f.write("%s\n" % url)
    
driver.quit()






Starting to get articles URLs from page:  RTVSLO.si - Najboljši splet vsebin
I'm on page: 1
1 https://www.rtvslo.si/zdravje/novi-koronavirus
2 https://www.rtvslo.si/slovenija/skupaj-naprej-v-premagovanje-epidemije/600811
3 https://www.rtvslo.si/slovenija/jansa-zavrnil-ocitke-da-vlada-ne-uposteva-stroke-morda-predlogov-ne-sprejme-isti-teden/600821
4 https://www.rtvslo.si/zdravje/novi-koronavirus/skupaj-naprej-z-verodostojnimi-informacijami-do-ustreznega-ravnanja-in-zmage-nad-epidemijo/600570
5 https://www.rtvslo.si/zdravje/novi-koronavirus/zaracunavanje-dodatnih-stroskov-ob-hitrem-testiranju-je-nezakonito/600686
6 https://www.rtvslo.si/zdravje/novi-koronavirus/po-evropi-rekordno-stevilo-okuzb-z-novim-koronavirusom/600593
7 https://www.rtvslo.si/zdravje/novi-koronavirus/poklukar-izvajajmo-ukrepe-in-ce-bomo-uspesni-zaprtja-druzbe-ne-bo/600574
8 https://www.rtvslo.si/slovenija/preberite-tudi/univerza-v-ljubljani-o-druzbenih-izzivih-v-postpandemicnem-casu/600555
9 https://www.rtvslo.si/radi

In [4]:
len(URLS)

91

In [1]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 7.29.0
ipykernel        : 6.4.1
ipywidgets       : 7.6.5
jupyter_client   : 7.0.1
jupyter_core     : 4.8.1
jupyter_server   : not installed
jupyterlab       : not installed
nbclient         : 0.5.3
nbconvert        : 6.1.0
nbformat         : 5.1.3
notebook         : 6.4.5
qtconsole        : 5.1.1
traitlets        : 5.1.0


## Web scraping

News portals generally provide a search functionality to search for news of interest. Your goal is to (a) programatically open a news site (rtvslo.si or 24ur.com), (b) enter query term `koronavirus`, (c) iterate through all the results and (d) parse each news article.

An example of Web pages from rtvslo.si:

<table>
    <tr>
        <td style="text-align: left;">rtvslo.si search results page</td>
        <td style="text-align: left;">rtvslo.si news article page</td>
    </tr>
    <tr>
        <td><img src="./rtvslo1.png" /></td>
        <td><img src="./rtvslo2.png" /></td>
    </tr>
</table>

Export the extracted data in a JSON format. Also define the schema of your data, for example as follows:

```
[
  {
    "type": "news",
    "author": "Ai. Ma.",
    "datetime_published": "20. oktober 2021 22:53 CET",
    "title": "Lejko Zupanc: Starost ljudi, ki potrebujejo najbolj invazivno podporo, se znižuje",
    "subtitle": "Pogovor s strokovnjakinjo v Odmevih",
    "headline": "Strokovnjaki se bojijo mrzlih in prazničnih dni, ko se bodo ljudje več družili v zaprtih prostorih in manj prezračeva...",
    "content": "Podatki o širjenju virusa niso dobri. Kakšne so trenutno razmere na vaši kliniki? 
Trenutne razmere so nekje na robu zmogljivosti, ki smo jih predvideli za covidne bolnike, tako v enoti intenzivne ter ...",
    "tags": ["Novi koronavirus", "Covid-19", "Tatjana Lejko Zupanc"]
    "comments": [
        {
            "user": "anonymous",
            "comment": "Upajmo, da bo vse ok ..."
        }, ...
    ]
    
  }, 
  {
    ...
]
```

Schema could be defined also using [JSON Schema guidelines](https://json-schema.org/).

`TODO: definition and short description of JSON data schema of the extracted data.`

Proposed structure of the implementation is below. You can also import code from the accompanying *.py* files in the repository, so the code is clearer or organize code completely by yourself.

In [ ]:
# Implement news article page parsing

def parse_news_article(url):
    ...
    return news_article_data

In [ ]:
# Implement search results page parsing 

def parse_search_results(url):
    ...
    return results, next_page_url

In [ ]:
# Implement parsing, data merging and final representation

def search():
    ...
    return data

In [ ]:
# Main program and data export into a JSON

MAIN_URL = "https://www.rtvslo.si"

data = search()
# Save data to UTF-8 encoded JSON
...

## Basic summarization

Prepare and show at least five basic visualizations of the extracted data as presented in the chapter *Summarizing data - the basics* of the course's e-book. Explain each visualization of the data.

In [ ]:
# Read data from JSON

data = ...

### Visualization 1

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 1 code

...

### Visualization 2

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 2 code

...

### Visualization 3

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 3 code

...

### Visualization 4

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 4 code

...

### Visualization 5

`TODO: name the visualization and describe it`

In [ ]:
# Visualization 5 code

...